In [175]:
import random
import numpy as np
class Layer:
    def __init__(self, width, prev_width, activation_fun, d_fun):
        self.width = width
        self.prev_width = prev_width
        self.W = self.__initializeWeights(width, prev_width)
        self.activation_fun = activation_fun
        self.weightedSums = np.zeros(prev_width + 1)
        self.activations = np.zeros(prev_width + 1)
        self.d_fun = d_fun
        
    def __initializeWeights(self, width, prev_width):
        W = np.random.rand(prev_width + 1, width)
        return W

    def propogate(self, inputs):
        inputs = np.append(1, inputs) # Prepend 1 to calc bias
        self.weightedSums = np.matmul(inputs, self.W)
        self.activations = [self.activation_fun(x) for x in self.weightedSums]
        
    def updateWeight(self, tail_neuron, head_neuron, deriv, step):
        self.W[tail_neuron][head_neuron] -= deriv * step
        
    def getWidth(self):
        return self.width
    
    def getWeights(self):
        return self.W
    
    def d_sigma(self, x):
        return self.d_fun(x)
    
    def getWeightedSums(self):
        return self.weightedSums
    
    def getActivations(self):
        return self.activations

In [178]:
class NeuralNet:
    def __init__(self, hiddenLayerWidths, activation_functions, d_functions, train_X, train_y, step):
        self.train_X = train_X
        self.train_y = train_y
        self.input_N = len(train_X[0])
        self.activation_funs = activation_functions
        self.d_funs = d_functions
        self.layers = self.__initializeLayers(hiddenLayerWidths)
        self.sampleLayerWeightedSums = [[[] for i in range(len(self.layers))] for x in range(len(self.train_X))]
        self.sampleLayerActivations = [[[] for i in range(len(self.layers))] for x in range(len(self.train_X))]
        self.step = step
        
    def __initializeLayers(self, hiddenLayerWidths):
        layers = []
        prev_width = self.input_N
        for width, activation_fun, derivative_fun in zip(hiddenLayerWidths, self.activation_funs, self.d_funs):
            layers.append(Layer(width, prev_width, activation_fun, derivative_fun))
            prev_width = width
        return layers
    
    def __computeLayers(self):
        for sample_n in range(len(self.train_X)):
            currInp = self.train_X[sample_n]
            for layer_l in range(len(self.layers)):
                self.layers[layer_l].propogate(currInp)
                self.sampleLayerWeightedSums[sample_n][layer_l] = self.layers[layer_l].getWeightedSums()
                activations = self.layers[layer_l].getActivations()
                self.sampleLayerActivations[sample_n][layer_l] = activations
                currInp = activations
    
    def getLoss(self): # Loss function: C = 1/2 * sum(||y - f(x)||^2)
        self.__computeLayers()
        loss = 0
        for sample_n in range(len(self.train_y)):
            diff = np.subtract(self.train_y[sample_n], self.sampleLayerActivations[sample_n][-1])
            loss += np.linalg.norm(diff) **2
        return loss / 2

    def getGradient(self):
        errors = [[0 for i in range(self.layers[x].getWidth())] for x in range(len(self.layers))]
        
        # Compute last layers error
        for sample_n in range(len(self.train_y)):
            a_grad = np.subtract(self.sampleLayerActivations[sample_n][-1], self.train_y[sample_n])
            d_sigma = [self.layers[-1].d_sigma(neuronSum) 
                       for neuronSum in self.sampleLayerWeightedSums[sample_n][-1]]
            errors[-1] = np.add(errors[-1], np.multiply(a_grad, d_sigma))
            
        # Compute other layers errors
        for layer_l in range(len(self.layers) - 2, -1, -1):
            left = np.matmul(self.layers[layer_l + 1].getWeights()[1:], errors[layer_l + 1])
            for sample_n in range(len(self.train_y)):
                right = [self.layers[layer_l].d_sigma(neuronSum)
                         for neuronSum in self.sampleLayerWeightedSums[sample_n][layer_l]]
            errors[layer_l] = np.add(errors[layer_l], np.multiply(left, right))
        
        activationSums = [[0 for i in range(self.layers[x].getWidth())] for x in range(len(self.layers))]
        
        # Compute sum of all neuron activations over all samples
        for sample_n in range(len(self.train_X)):
            for layer_l in range(len(self.layers)):
                for neuron_n in range(self.layers[layer_l].getWidth()):
                    activationSums[layer_l] = np.add(activationSums[layer_l], self.sampleLayerActivations[sample_n][layer_l])
        
        # Preupdate first layer
        firstActivationSum = [0 for x in range(self.input_N)]
        for i in range(len(self.train_X)):
            firstActivationSum = np.add(self.train_X[i], firstActivationSum)
            
        for neuron_n in range(self.layers[0].getWidth()):
            error = errors[0][neuron_n]
            self.layers[0].updateWeight(0, neuron_n, error, self.step)
            for inc_n in range(1, self.input_N + 1):
                self.layers[0].updateWeight(inc_n, neuron_n, error * firstActivationSum[inc_n - 1], self.step)

        # Update rest of layer
        for layer_l in range(1, len(self.layers)):
            for neuron_n in range(self.layers[layer_l].getWidth()):
                error = errors[layer_l][neuron_n]
                self.layers[layer_l].updateWeight(0, neuron_n, error, self.step)
                for inc_n in range(1, self.layers[layer_l - 1].getWidth() + 1):
                    self.layers[layer_l].updateWeight(inc_n, neuron_n, error * activationSums[layer_l - 1][inc_n - 1], self.step)
                

In [187]:
def f(x):
    return max(0, x)

def df(x):
    if x <= 0:
        return 0
    else:
        return 1
    
nn = NeuralNet([50, 50, 2], [f, f, f], [df, df, df], [[20, 10 ,40], [30, 15, 15], [15, 25, 24]], [[800,200], [856, 400], [900, 444]], 0.000000000005)

print(nn.getLoss())
for i in range(500):
    nn.getGradient()
    print(nn.getLoss())


1391679048.821583
1149478489.7229633
952612209.0076916
791645961.6862352
659386883.9527857
550271777.7876389
459944433.8723171
384957315.16377264
322556948.8935373
270526842.5671163
227070683.46050406
190724241.5091731
160288054.3758148
134775384.4086815
113371553.92049961
95401867.01708782
80306088.44316955
67617984.70881394
56948812.99720853
47973917.13834581
40421789.46294148
34065104.461024344
28713339.846958157
24206683.281902906
20410985.92814266
17213572.385623075
14519754.098666672
12249922.703184392
10337122.971237043
8725023.445159538
7366217.612732071
6220800.365550054
5255174.116837338
4441046.800487836
3754590.3911218406
3175733.8562057223
2687568.7959664455
2275849.6188570014
1928573.0780656566
1635624.4688211952
1388479.8463054341
1179955.342168353
1003996.0928327849
855498.493128965
730160.4938485451
624355.5041265961
535026.1660682373
459594.8620416531
395888.3130908774
342074.0454020664
296606.85354508244
258183.68504601193
225705.61970441623
198245.82646756808
175022

44807.96814268797
44807.968142687976
44807.968142687976
44807.968142688
44807.96814268797
44807.96814268793
44807.96814268792
44807.96814268791
44807.96814268791
44807.96814268795
44807.96814268794
44807.96814268794
44807.96814268797
44807.96814268795
44807.96814268793
44807.96814268792
44807.968142687874
44807.96814268788
44807.96814268786
44807.968142687845
44807.96814268786
44807.968142687874
44807.96814268787
44807.968142687874
44807.968142687874
44807.96814268789
44807.96814268791
44807.96814268791
44807.9681426879
44807.968142687896
44807.96814268791
44807.96814268788
44807.96814268789
44807.96814268787
44807.968142687896
44807.968142687896
44807.96814268789
44807.96814268788
44807.9681426879
44807.96814268787
44807.968142687874
44807.968142687874
44807.96814268784
44807.96814268784
44807.96814268784
44807.968142687874
44807.96814268786
44807.96814268785
44807.96814268785
44807.96814268785


In [180]:
print(list(range(2 - 2, -1, -1)))

[0]


In [52]:
a = [1, 2, 3]
b = [4, 5, 6]
np.matmul(a,b)

32

In [56]:
layer = Layer(3, 4, f, df)
layer.getWeights()

array([[0.58049094, 0.54493143, 0.25574428],
       [0.33914321, 0.8644203 , 0.39340868],
       [0.79962713, 0.47052293, 0.0560082 ],
       [0.1487162 , 0.66520349, 0.89362601]])

In [57]:
layer.getWeights()

array([[0.58049094, 0.54493143, 0.25574428],
       [0.33914321, 0.8644203 , 0.39340868],
       [0.79962713, 0.47052293, 0.0560082 ],
       [0.1487162 , 0.66520349, 0.89362601]])

In [73]:
list(range(1, 6))

[1, 2, 3, 4, 5]

In [163]:
a = [1, 2, 3]
b = ['a', 'b', 'c']
c = ['1', '2', '3']

list(zip(a, b, c))

[(1, 'a', '1'), (2, 'b', '2'), (3, 'c', '3')]